# Load data

In [0]:
dbutils.fs.mount(
  source = "wasbs://bronze@ilab9788543873.blob.core.windows.net",
  mount_point = "/mnt/ilab9788543873/bronze",
  extra_configs = {"fs.azure.account.key.ilab9788543873.blob.core.windows.net":"TAyrrfaiCQ86vkgSXyStufowrMVdk4T45mVw6TNcFFJocR6pTXy6ZMSUUTeeh5FpNTsBVAbVqdwk+AStHA3x1g=="})

In [0]:
import pandas as pd
import json as json

# Define the file path
file_path = "dbfs:/mnt/ilab9788543873/bronze/raw_output.json"

# 1. Read the JSON file into a DataFrame
# Load JSON data
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        data_json = json.load(file)
    # Indicating that the data was loaded successfully
    load_success = True
except json.JSONDecodeError as e:
    load_success = False
    load_error = str(e)


In [0]:
df = pd.DataFrame(data_json)

def find_non_primitive_columns(df):
    non_primitive_columns = []
    for col in df.columns:
        # Find the first non-null entry in the column
        non_null_entries = df[col].dropna().values
        if len(non_null_entries) > 0:
            first_non_null = non_null_entries[0]
            # Check if it's a list or dictionary
            if isinstance(first_non_null, (list, dict)):
                non_primitive_columns.append(col)
    return non_primitive_columns

# Example usage:
non_primitive_cols = find_non_primitive_columns(df)
print("Columns containing lists or dictionaries:", non_primitive_cols)


Columns containing lists or dictionaries: ['biblio', 'legal_status', 'abstract', 'claims', 'description']


# 1.0 Abstract

In [0]:
import pandas as pd

# Load the dataset
data = pd.read_json(file_path)

abstract_df = data[['lens_id', 'abstract']].copy()

def extract_text(abstract_item):
    # If the item is a dictionary, return the 'text' value
    if isinstance(abstract_item, dict):
        return abstract_item.get('text', None)  # Use .get() to safely retrieve the key
    # If the item is a list, you can decide how you want to handle this.
    # For example, you can return the first item's 'text' value.
    elif isinstance(abstract_item, list) and len(abstract_item) > 0:
        return abstract_item[0].get('text', None)
    else:
        return None

abstract_df['text'] = abstract_df['abstract'].apply(extract_text)
abstract_df = abstract_df.drop(columns=['abstract'])


# 2.0 Bibliographic Application Reference:

In [0]:
biblio_app_ref_df = data[['lens_id']].copy()
biblio_app_ref = data['biblio'].apply(lambda x: x.get('application_reference', {})).apply(pd.Series)

# Adding them to our DataFrame
biblio_app_ref_df = pd.concat([biblio_app_ref_df, biblio_app_ref], axis=1)


In [0]:
biblio_app_ref_df.head()

,lens_id,jurisdiction,doc_number,kind,date
0,041-096-994-434-303,US,201615237340,A,2016-08-15
1,018-076-028-631-459,US,201715489040,A,2017-04-17
2,086-912-788-458-991,US,201715858486,A,2017-12-29
3,044-976-460-122-895,US,201715788948,A,2017-10-20
4,123-105-483-633-575,US,201715430408,A,2017-02-10


# 3.0 Bibliographic Cited By:


In [0]:
biblio_cited_by_df = data[['lens_id']].copy()
biblio_cited_by = data['biblio'].apply(lambda x: x.get('cited_by', {})).apply(pd.Series)
biblio_cited_by_df['patent_count'] = biblio_cited_by['patent_count']

# Handle the 'patents' list
cited_by_patents = biblio_cited_by['patents'].apply(pd.Series).stack().reset_index(level=1, drop=True)
cited_by_patents_df = pd.DataFrame(cited_by_patents.tolist(), index=cited_by_patents.index)
cited_by_patents_df = cited_by_patents_df.rename(columns={
    'lens_id': 'cited_lens_id', 
    'document_id': 'cited_document_id'
})

# Combine lens_id with the patents data
biblio_cited_by_df = biblio_cited_by_df.join(cited_by_patents_df, how='outer')

print(biblio_cited_by_df.head())


               lens_id  patent_count  \
0  041-096-994-434-303          14.0   
0  041-096-994-434-303          14.0   
0  041-096-994-434-303          14.0   
0  041-096-994-434-303          14.0   
0  041-096-994-434-303          14.0   

                                   cited_document_id        cited_lens_id  
0  {'jurisdiction': 'US', 'doc_number': '11416084...  101-652-338-808-281  
0  {'jurisdiction': 'US', 'doc_number': '11409376...  095-034-701-034-046  
0  {'jurisdiction': 'US', 'doc_number': '11630836...  003-580-552-488-28X  
0  {'jurisdiction': 'US', 'doc_number': '11734706...  044-305-578-212-354  
0  {'jurisdiction': 'US', 'doc_number': '11010940...  194-490-311-975-877  


# 3.1 cited_document_id

In [0]:
# Extracting nested columns from the 'cited_document_id'
cited_document_id_df = biblio_cited_by_df['cited_document_id'].apply(pd.Series)

# Rename columns for clarity
cited_document_id_df = cited_document_id_df.rename(columns={
    'jurisdiction': 'cited_jurisdiction',
    'doc_number': 'cited_doc_number',
    'kind': 'cited_kind'
})

# Drop the original 'cited_document_id' column
biblio_cited_by_df = biblio_cited_by_df.drop(columns=['cited_document_id'])

# Join the flattened columns to the original DataFrame
biblio_cited_by_df = pd.concat([biblio_cited_by_df, cited_document_id_df], axis=1)


# 4.0 classifications

In [0]:
# Extract the classifications list and expand it into its own DataFrame
classifications_df = data['biblio'].apply(lambda x: x.get('classifications_cpc', {}).get('classifications', {})).explode().reset_index(drop=True)

# Extract the 'symbol' from the classifications_df
classifications_df = classifications_df.apply(lambda x: x.get('symbol') if isinstance(x, dict) else None)

# Adding lens_id for reference
classifications_df = pd.concat([data['lens_id'], classifications_df], axis=1)
classifications_df.columns = ['lens_id', 'symbol']



# 5.0 invention_title

In [0]:
# Initial DataFrame with lens_id
invention_title_df = data[['lens_id']].copy()

# Extracting the 'invention_title' from the nested dictionaries
titles = data['biblio'].apply(lambda x: x.get('invention_title', [{}]))

# Expanding the lists to rows and extracting the text
exploded_titles = titles.explode().apply(lambda x: x.get('text') if isinstance(x, dict) else None)

# Combine the lens_id and exploded titles
invention_title_df = pd.concat([invention_title_df.reset_index(drop=True), exploded_titles.reset_index(drop=True)], axis=1)

# Rename the columns
invention_title_df.columns = ['lens_id', 'invention_title']

# Remove rows where 'invention_title' is None or NaN
invention_title_df = invention_title_df.dropna(subset=['invention_title']).reset_index(drop=True)


# 6.1 applicants

In [0]:
# Initial DataFrame with lens_id
applicants_df = data[['lens_id']].copy()

# Extracting the 'applicants' from the nested dictionaries
applicants = data['biblio'].apply(lambda x: x.get('parties', {}).get('applicants', [{}]))

# Further extracting 'extracted_name' and its 'value'
applicants_names = applicants.apply(lambda x: [y.get('extracted_name', {}).get('value') for y in x if isinstance(y, dict) and 'extracted_name' in y])

# Expanding the lists to rows
exploded_applicants = applicants_names.explode()

# Combine the lens_id and exploded applicant names
applicants_df = pd.concat([applicants_df.reset_index(drop=True), exploded_applicants.reset_index(drop=True)], axis=1)

# Rename the columns
applicants_df.columns = ['lens_id', 'applicant_name']

# Remove rows where 'applicant_name' is None or NaN
applicants_df = applicants_df.dropna(subset=['applicant_name']).reset_index(drop=True)


In [0]:
applicants_df.head()

,lens_id,applicant_name
0,041-096-994-434-303,ALEGEUS TECH LLC
1,018-076-028-631-459,INTEL CORP
2,086-912-788-458-991,INTEL CORP
3,044-976-460-122-895,GOOGLE LLC
4,123-105-483-633-575,QUALCOMM INC


# 6.2 inventors

In [0]:
# Initial DataFrame with lens_id
inventors_df = data[['lens_id']].copy()

# Extracting the 'inventors' from the nested dictionaries
inventors = data['biblio'].apply(lambda x: x.get('parties', {}).get('inventors', [{}]))

# Expanding the lists to rows and extracting the name value
exploded_inventors = inventors.explode().apply(lambda x: x.get('extracted_name', {}).get('value') if isinstance(x, dict) else None)

# Combine the lens_id and exploded inventors
inventors_df = pd.concat([inventors_df.reset_index(drop=True), exploded_inventors.reset_index(drop=True)], axis=1)

# Rename the columns
inventors_df.columns = ['lens_id', 'inventor_name']

# Remove rows where 'inventor_name' is None or NaN
inventors_df = inventors_df.dropna(subset=['inventor_name']).reset_index(drop=True)


# 6.3 owners_all

In [0]:
# Initial DataFrame with lens_id
owners_all_df = data[['lens_id']].copy()

# Extracting the 'owners_all' from the nested dictionaries
owners_all = data['biblio'].apply(lambda x: x.get('parties', {}).get('owners_all', [{}]))

# Expanding the lists to rows and extracting the name value, address, country, execution_date, and recorded_date
exploded_owners_all = owners_all.explode().apply(lambda x: {
    'owner_name': x.get('extracted_name', {}).get('value'),
    'address': x.get('extracted_address'),
    'country': x.get('extracted_country'),
    'execution_date': x.get('execution_date'),
    'recorded_date': x.get('recorded_date')
} if isinstance(x, dict) else {})

# Convert this Series of dictionaries into a DataFrame
owners_all_details_df = pd.DataFrame(exploded_owners_all.tolist())

# Combine the lens_id and exploded owners_all details
owners_all_df = pd.concat([owners_all_df.reset_index(drop=True), owners_all_details_df.reset_index(drop=True)], axis=1)

# Remove rows where all owner details are NaN or None
owners_all_df = owners_all_df.dropna(subset=['owner_name', 'address', 'country', 'execution_date', 'recorded_date'], how='all').reset_index(drop=True)


# 7.1 priority_claims

In [0]:
# Initial DataFrame with lens_id
priority_claims_df = data[['lens_id']].copy()

# Extracting the 'claims' from the nested dictionaries within 'priority_claims'
claims = data['biblio'].apply(lambda x: x.get('priority_claims', {}).get('claims', [{}]))

# Expanding the lists to rows
exploded_claims = claims.explode()

# Convert this Series of dictionaries into a DataFrame
claims_details_df = pd.DataFrame(exploded_claims.tolist())

# Combine the lens_id and exploded claims details
priority_claims_df = pd.concat([priority_claims_df.reset_index(drop=True), claims_details_df.reset_index(drop=True)], axis=1)

# Remove rows where all claim details are NaN or None
priority_claims_df = priority_claims_df.dropna(subset=['date', 'doc_number', 'jurisdiction', 'kind', 'sequence'], how='all').reset_index(drop=True)


# 7.2 earliest_claims

In [0]:
# Initial DataFrame with lens_id
earliest_claim_df = data[['lens_id']].copy()

# Extracting the 'earliest_claim' from the nested dictionaries within 'priority_claims'
earliest_claim = data['biblio'].apply(lambda x: x.get('priority_claims', {}).get('earliest_claim', {}))

# Convert this Series of dictionaries into a DataFrame
earliest_claim_details_df = pd.DataFrame(earliest_claim.tolist())

# Combine the lens_id and earliest_claim details
earliest_claim_df = pd.concat([earliest_claim_df.reset_index(drop=True), earliest_claim_details_df.reset_index(drop=True)], axis=1)

# Remove rows where 'date' is NaN or None
earliest_claim_df = earliest_claim_df.dropna(subset=['date']).reset_index(drop=True)


# 8.0 publication_reference 

In [0]:
# Initial DataFrame with lens_id
publication_reference_df = data[['lens_id']].copy()

# Extracting the 'publication_reference' from the nested dictionaries within 'biblio'
pub_ref = data['biblio'].apply(lambda x: x.get('publication_reference', {}))

# Convert this Series of dictionaries into a DataFrame
pub_ref_details_df = pd.DataFrame(pub_ref.tolist())

# Combine the lens_id and pub_ref details
publication_reference_df = pd.concat([publication_reference_df.reset_index(drop=True), pub_ref_details_df.reset_index(drop=True)], axis=1)

# Remove rows where all publication reference details are NaN or None
publication_reference_df = publication_reference_df.dropna(subset=['date', 'doc_number', 'jurisdiction', 'kind'], how='all').reset_index(drop=True)


# 9.0 reference_cited

In [0]:
# Initial DataFrame with lens_id
references_cited_df = data[['lens_id']].copy()

# Extracting 'citations' from the nested dictionaries within 'references_cited'
citations = data['biblio'].apply(lambda x: x.get('references_cited', {}).get('citations', [{}]))

# Explode the citations list
exploded_citations = citations.explode().reset_index(drop=True)

# Extract cited_phase, sequence, nplcit details
references_cited_df['cited_phase'] = exploded_citations.apply(lambda x: x.get('cited_phase'))
references_cited_df['sequence'] = exploded_citations.apply(lambda x: x.get('sequence'))
references_cited_df['nplcit_lens_id'] = exploded_citations.apply(lambda x: x.get('nplcit', {}).get('lens_id'))
references_cited_df['nplcit_text'] = exploded_citations.apply(lambda x: x.get('nplcit', {}).get('text'))

# Extract details from patcit and expand patcit's document_id
patcit_document_id = exploded_citations.apply(lambda x: x.get('patcit', {}).get('document_id', {})).apply(pd.Series)

# Rename columns for clarity
patcit_document_id.columns = ['patcit_' + col for col in patcit_document_id.columns]

# Combine the DataFrames
references_cited_df = pd.concat([references_cited_df.reset_index(drop=True), patcit_document_id.reset_index(drop=True)], axis=1)

# Filter rows with no relevant details
references_cited_df = references_cited_df.dropna(subset=['cited_phase', 'nplcit_lens_id', 'sequence'], how='all').reset_index(drop=True)


# 10.0 claims

In [0]:
# Filter out non-list entries
filtered_data = data[data['claims'].apply(lambda x: isinstance(x, list))]

# Using a nested list comprehension to extract claim_texts
claim_texts = filtered_data['claims'].apply(lambda claims: [claim.get('claim_text', [None])[0] for claim in claims if isinstance(claim, dict)])

# Add lens_id and explode the lists to rows
claims_df = pd.DataFrame({
    'lens_id': filtered_data['lens_id'].repeat(claim_texts.apply(len)),
    'claim_text': [text for texts in claim_texts for text in texts]
})

# Remove rows where 'claim_text' is None or NaN
claims_df = claims_df.dropna(subset=['claim_text']).reset_index(drop=True)


In [0]:
# Extracting the 'claim_text' from the nested structure
def extract_claim_texts(claims_list):
    if not isinstance(claims_list, list):
        return []
    claim_texts = []
    for entry in claims_list:
        if 'claims' in entry:
            for claim in entry['claims']:
                if 'claim_text' in claim:
                    claim_texts.extend(claim['claim_text'])
    return claim_texts

data['claim_texts'] = data['claims'].apply(extract_claim_texts)

# Now, we'll explode the claim_texts to have one row per claim_text
exploded_data = data.explode('claim_texts')[['lens_id', 'claim_texts']]
exploded_data = exploded_data.rename(columns={'claim_texts': 'claim_text'})

# Checking the structure of the resulting dataframe
claims_df = exploded_data


# 11.0 description

In [0]:
# Extracting the 'text' from the 'description' column
def extract_description_text(description):
    if isinstance(description, dict) and 'text' in description:
        return description['text']
    return None

data['description_text'] = data['description'].apply(extract_description_text)

# Now, you'll have a new column named 'description_text' containing the descriptions
descriptions_df = data[['lens_id', 'description_text']]


# 12.0 legal_status

In [0]:
# Extracting the 'grant_date', 'granted', and 'patent_status' from the 'legal_status' column
def extract_grant_date(legal_status):
    if isinstance(legal_status, dict) and 'grant_date' in legal_status:
        return legal_status['grant_date']
    return None

def extract_granted(legal_status):
    if isinstance(legal_status, dict) and 'granted' in legal_status:
        return legal_status['granted']
    return None

def extract_patent_status(legal_status):
    if isinstance(legal_status, dict) and 'patent_status' in legal_status:
        return legal_status['patent_status']
    return None

data['grant_date'] = data['legal_status'].apply(extract_grant_date)
data['granted'] = data['legal_status'].apply(extract_granted)
data['patent_status'] = data['legal_status'].apply(extract_patent_status)

# Now, you'll have three new columns named 'grant_date', 'granted', and 'patent_status'
legal_status_df = data[['lens_id', 'grant_date', 'granted', 'patent_status']]


# 13.0 df_main

In [0]:
df_main = data[['lens_id','date_published', 'doc_number', 'jurisdiction', 'kind', 'lang']]

# Compile

In [0]:
# Starting with df_main as the base
merged_df = df_main

# List of all dataframes to merge
dfs_to_merge = [abstract_df, biblio_cited_by_df, classifications_df, invention_title_df, 
                applicants_df, inventors_df, owners_all_df, priority_claims_df, 
                earliest_claim_df, publication_reference_df, references_cited_df, 
                claims_df, descriptions_df, legal_status_df]

# Iteratively merge dataframes on lens_id
for df in dfs_to_merge:
    merged_df = pd.merge(merged_df, df, on='lens_id', how='left')



# Load to Azure

In [0]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

# Your Azure Blob Storage credentials
account_name = 'ilab9788543873'
account_key = 'TAyrrfaiCQ86vkgSXyStufowrMVdk4T45mVw6TNcFFJocR6pTXy6ZMSUUTeeh5FpNTsBVAbVqdwk+AStHA3x1g=='
container_name = 'silver'

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# List of dataframes with their respective names
dfs = {
    'main': df_main,
    'abstract': abstract_df,
    'biblio_cited_by': biblio_cited_by_df,
    'classifications': classifications_df,
    'invention_title': invention_title_df,
    'applicants': applicants_df,
    'inventors': inventors_df,
    'owners_all': owners_all_df,
    'priority_claims': priority_claims_df,
    'earliest_claim': earliest_claim_df,
    'publication_reference': publication_reference_df,
    'references_cited': references_cited_df,
    'claims': claims_df,
    'descriptions': descriptions_df,
    'legal_status': legal_status_df
}

# Loop through each DataFrame and upload it to Azure Blob Storage
for blob_name, df in dfs.items():
    # Convert the DataFrame to a Parquet byte stream
    buffer = io.BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)
    
    # Create a blob client and upload the byte stream
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=f"{blob_name}.parquet")
    
    # Upload data
    blob_client.upload_blob(buffer.getvalue(), overwrite=True, blob_type="BlockBlob")
    print(f"{blob_name}.parquet uploaded to Azure Blob Storage")
